In [ ]:
# Classify the CIFAR100 dataset

import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import TensorDataset, DataLoader
from torchvision import datasets
from torchvision.transforms import transforms
import torch.optim as optim
import torchvision
import matplotlib.pyplot as plt

In [ ]:
batch_size = 64

train_dataset = datasets.CIFAR100(root='data', train=True, download=True)

DATA_MEANS = (train_dataset.data / 255.0).mean(axis=(0, 1, 2))
DATA_STD = (train_dataset.data / 255.0).std(axis=(0, 1, 2))

train_transform = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(DATA_MEANS, DATA_STD)
])

test_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(DATA_MEANS, DATA_STD)
])

train_set = datasets.CIFAR100(root='data', train=True, transform=train_transform, download=True)
test_set = datasets.CIFAR100(root='data', train=False, transform=test_transform, download=True)

train_dataset = datasets.CIFAR100(root='data', train=True, transform=train_transform, download=True)
val_dataset = datasets.CIFAR100(root='data', train=True, transform=test_transform, download=True)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_set, batch_size=batch_size, shuffle=False)

In [ ]:
class Model(nn.Module):
    def __init__(self):
        super(Model, self).__init__()
        self.conv1 = nn.Conv2d(3, 10, kernel_size=(3, 3), padding=1)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=(3, 3), padding=1)
        self.dropout_2d = nn.Dropout2d(p=0.25)
        self.fc1 = nn.Linear(7 * 7 * 20, 128)
        self.dropout = nn.Dropout(p=0.5)
        self.fc2 = nn.Linear(128, 100)
    
    def forward(self, x):
        x = self.dropout_2d(F.max_pool2d(self.conv1(x), kernel_size=2))
        x = self.dropout_2d(F.max_pool2d(self.conv2(x), kernel_size=2))
        x = x.view(-1, 7 * 7 * 20)
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)
    
    def reset_parameters(self):
        self.conv1.reset_parameters()
        self.conv2.reset_parameters()
        self.fc1.reset_parameters()
        self.fc2.reset_parameters()

In [ ]:
class Model(nn.Module):
    def __init__(self):
        super(Model, self).__init__()
        self.start = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=False)
        )

        self.next_layer = nn.Sequential(
              nn.Conv2d(128, 128, kernel_size=3, stride=2, padding=1),
              nn.BatchNorm2d(128),
              nn.ReLU(inplace=False)
)

        self.inception1 = Inception_Block(64, 32, 64, 64, 64, 64, 32)
        self.skip1 = Skip_Block(dense=False, a=True, in_channels=192, out1=64, out2=64, out3=128)
        self.inception2 = Inception_Block(128, 64, 128, 128, 128, 128, 64)
        self.skip2 = Skip_Block(dense=True, a=False, in_channels=384, out1=128, out2=128, out3=256)
        self.inception3 = Inception_Block(256, 128, 256, 256, 256, 256, 128)
        self.skip3 = Skip_Block(dense=True, a=True, in_channels=768, out1=256, out2=256, out3=512)
        self.pool = nn.AdaptiveAvgPool2d((1, 1))
        self.dropout = nn.Dropout(p=0.4)
        self.fc = nn.Sequential(
            nn.Linear(1024, 256),
            nn.ReLU(inplace=False),
            nn.Linear(256, 100)
        )

    def forward(self, x):
        x = self.start(x)
        x = self.inception1(x)
        x = self.skip1(x)
        x = self.next_layer(x)
        x = self.inception2(x)
        x = self.skip2(x)
        x = self.pool(x)
        x = self.inception3(x)
        x = self.skip3(x)
        x = x.view(x.shape[0], -1)
        x = self.dropout(x)
        x = self.fc(x)
        return x

class Inception_Block(nn.Module):
    def __init__(self, in_channels, out_one, out_two_prog, out_two, out_three_prog, out_three, out_four):
        super(Inception_Block, self).__init__()

        self.band1 = nn.Sequential(
            nn.Conv2d(in_channels, out_one, kernel_size=1),
            nn.ReLU(inplace=False)
        )

        self.band2 = nn.Sequential(
            nn.Conv2d(in_channels, out_two_prog, kernel_size=1),
            nn.ReLU(inplace=False),
            nn.Conv2d(out_two_prog, out_two, kernel_size=3, padding=1),
            nn.ReLU(inplace=False)
        )

        self.band3 = nn.Sequential(
            nn.Conv2d(in_channels, out_three_prog, kernel_size=1),
            nn.ReLU(inplace=False),
            nn.Conv2d(out_three_prog, out_three, kernel_size=3, padding=1),
            nn.ReLU(inplace=False)
        )

        self.band4 = nn.Sequential(
            nn.MaxPool2d(kernel_size=3, stride=1, padding=1, ceil_mode=True),
            nn.ReLU(inplace=False),
            nn.Conv2d(in_channels, out_four, kernel_size=1),
            nn.ReLU(inplace=False)
        )

    def forward(self, x):
        out1 = self.band1(x)
        out2 = self.band2(x)
        out3 = self.band3(x)
        out4 = self.band4(x)

        output = torch.cat([out1, out2, out3, out4], dim=1)
        return output

class Skip_Block(nn.Module):
    def __init__(self, dense, a, in_channels, out1, out2, out3):
        super(Skip_Block, self).__init__()
        self.a = a
        self.dense = dense
        self.path = nn.Sequential(
            nn.Conv2d(in_channels, out1, kernel_size=1),
            nn.BatchNorm2d(out1),
            nn.ReLU(inplace=False),
            nn.Conv2d(out1, out2, kernel_size=3, padding=1),
            nn.BatchNorm2d(out2),
            nn.ReLU(inplace=False),
            nn.Conv2d(out2, out3, kernel_size=1),
            nn.BatchNorm2d(out3)
        )

        self.base = nn.Sequential(nn.Conv2d(in_channels, out3, kernel_size=1),
                             nn.BatchNorm2d(out3)
                             )

        self.setup = nn.Sequential(
            nn.Conv2d(in_channels, out3, kernel_size=1),
            nn.BatchNorm2d(out3)
        )


    def forward(self, x):
        if self.dense:
            if self.a:
                out = torch.cat([self.path(x), self.base(x)], dim=1)
            else:
                out = torch.cat([self.path(x) + self.base(x)], dim=1)
            return out
        else:
          if self.a:
              out = self.path(x) + self.base(x)
          else:
              out = self.path(x) + self.setup(x)
          out = F.relu(out)
          return out



In [ ]:

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = Model()
model = model.to(device)

epochs = 100
lr = 0.001
weight_decay = 1e-4
optimizer = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay)
loss = nn.CrossEntropyLoss()
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer=optimizer, T_max=3, last_epoch=-1)

def train():

    for epoch in range(epochs):

        model.train()
        running_loss = 0.
        last_loss = 0.

        for i, data in enumerate(train_loader):
            inputs, labels = data

            inputs = inputs.to(device)
            labels = labels.to(device)

            optimizer.zero_grad()

            outputs = model(inputs)

            l = loss(outputs, labels)
            l.backward()
            optimizer.step()
            running_loss += l.item()
        running_loss = running_loss / len(train_loader)
        print('Epoch: ' + str(epoch) + ' Loss: ' + str(running_loss))
        scheduler.step()


    return last_loss



In [ ]:
def test():
    model.eval()
    test_loss = 0.0
    correct = 0
    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)

            outputs = model(inputs)
            l = loss(outputs, labels)
            test_loss += l.item()

            pred = outputs.argmax(dim=1)
            correct += pred.eq(labels).sum().item()

    accuracy = 100.0 * correct / len(test_loader.dataset)
    return test_loss / len(test_loader), accuracy

In [ ]:
for epoch in range(0, epochs):
    train_loss = train()
    test_loss, test_accuracy = test()

    print(f"Epoch {epoch}/{epochs}:")
    print(f"  Train Loss: {train_loss:.4f}")
    print(f"  Test Loss: {test_loss:.4f}")
    print(f"  Test Accuracy: {test_accuracy:.2f}%")
